In [16]:
import pandas as pd

In [ ]:
df_Renewable = pd.read_csv('../Dataset/한국전력거래소_인천시_태양광_풍력_발전량_2024.csv', encoding='cp949')
df_climate = pd.read_csv('../Dataset/기후데이터_인천시_2024.csv', encoding='cp949')=

In [18]:
df_Renewable

,거래일자,거래시간,지역,연료원,전력거래량(MWh)
0,2024-01-01,1,인천시,태양광,0.000000
1,2024-01-01,2,인천시,태양광,0.000000
2,2024-01-01,3,인천시,태양광,0.000000
3,2024-01-01,4,인천시,태양광,0.000000
4,2024-01-01,5,인천시,태양광,0.000000
...,...,...,...,...,...
8779,2024-12-31,20,인천시,태양광,2.215681
8780,2024-12-31,21,인천시,태양광,1.068835
8781,2024-12-31,22,인천시,태양광,0.288480
8782,2024-12-31,23,인천시,태양광,0.000080


In [19]:
df_climate

,지점,지점명,일시,기온(°C),기온 QC플래그,강수량(mm),강수량 QC플래그,풍속(m/s),풍속 QC플래그,풍향(16방위),...,최저운고(100m ),시정(10m),지면상태(지면상태코드),현상번호(국내식),지면온도(°C),지면온도 QC플래그,5cm 지중온도(°C),10cm 지중온도(°C),20cm 지중온도(°C),30cm 지중온도(°C)
0,112,인천,2024-01-01 00:00,0.7,NaN,NaN,9.0,0.6,NaN,50.0,...,NaN,717.0,NaN,19.0,0.0,NaN,-0.7,0.2,1.0,1.8
1,112,인천,2024-01-01 01:00,0.6,NaN,NaN,NaN,1.1,NaN,70.0,...,NaN,606.0,NaN,19.0,-0.1,NaN,-0.7,0.2,1.0,1.7
2,112,인천,2024-01-01 02:00,0.3,NaN,NaN,NaN,1.8,NaN,360.0,...,13.0,306.0,NaN,19.0,-0.1,NaN,-0.7,0.3,0.9,1.7
3,112,인천,2024-01-01 03:00,0.5,NaN,NaN,NaN,0.7,NaN,320.0,...,9.0,479.0,NaN,19.0,-0.1,NaN,-0.7,0.3,1.0,1.8
4,112,인천,2024-01-01 04:00,0.4,NaN,NaN,NaN,1.5,NaN,70.0,...,10.0,381.0,NaN,19.0,-0.2,NaN,-0.7,0.3,1.0,1.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8779,112,인천,2024-12-31 19:00,0.1,NaN,NaN,NaN,1.7,NaN,290.0,...,NaN,4861.0,NaN,NaN,0.1,NaN,1.3,2.2,2.6,3.1
8780,112,인천,2024-12-31 20:00,-0.1,NaN,NaN,NaN,2.2,NaN,270.0,...,NaN,4599.0,NaN,NaN,-0.2,NaN,1.1,2.0,2.5,3.2
8781,112,인천,2024-12-31 21:00,0.1,NaN,NaN,NaN,2.4,NaN,270.0,...,NaN,4925.0,NaN,NaN,-0.4,NaN,0.7,1.9,2.6,3.2
8782,112,인천,2024-12-31 22:00,0.3,NaN,NaN,NaN,2.3,NaN,290.0,...,NaN,4779.0,NaN,NaN,-0.5,NaN,0.8,1.7,2.5,3.2
